## Capstone Project - IMDB Dataset
### Phase 1 & 2
### By: Amita Saini


### Phase 1

1) Webscrape the provided URL  - IMDB dataset:
https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1

2) Store the entire data in two different CSV files as per the given fields:

The first CSV file data contains :
Sno, Movie Name, Director Name, Duration, year, ratings, Metascore
Bifurcate the Director field into subfields as per the number of directors of the movie belongs to such as Director1, director2

The second CSV file contains the following:
Movie Name, stars, votes, Genre, Gross collection, popularity, Certification
Bifurcate the stars field into 4 subfields as per the number of stars worked in the movie such as star1, star2, star3, star4
Bifurcate the genre into 3 subfields as per the number of genres the movie belongs to such as :
Genre1, genre2, genre3

In [1]:
# Import necessary Libraries

from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [2]:

url = 'https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1'

# List to store data for the first CSV
first_csv_data = []

# List to store data for the second CSV
second_csv_data = []

page = 1
while True:
    # Send a GET request to the URL
    response = requests.get(url + f'&start={page}')

    # Create a BeautifulSoup object from the response content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all the movie containers
    movie_containers = soup.find_all('div', class_='lister-item mode-advanced')

    # If no movie containers are found, exit the loop
    if not movie_containers:
        break

    # Iterate over each movie container
    for container in movie_containers:
        # Extract data for the first CSV
        title = container.h3.a.text.strip()
        directors = container.find('p', class_='').find_all('a')
        director_names = [director.text.strip() for director in directors]
        duration = container.find('span', class_='runtime').text.strip().split()[0]
        year = container.find('span', class_='lister-item-year').text.strip('()')
        rating = container.find('strong').text.strip()
        metascore = container.find('span', class_='metascore').text.strip() if container.find('span', class_='metascore') else ''

        first_csv_data.append({
            'Sno': len(first_csv_data) + 1,
            'Movie Name': title,
            'Director Name': ', '.join(director_names),
            'Duration': duration,
            'Year': year,
            'Ratings': rating,
            'Metascore': metascore
        })

        # Extract data for the second CSV
        stars = container.find('p', class_='').find_all('a')[1:]
        star_names = [star.text.strip() for star in stars]
        votes = container.find('span', attrs={'name': 'nv'})['data-value']
        genres = container.find('span', class_='genre').text.strip().split(', ')
        gross = container.find('p', class_='sort-num_votes-visible').find_all('span')[-1]['data-value'] if container.find('p', class_='sort-num_votes-visible').find_all('span') else ''
        popularity = container.find('div', class_='lister-item-content').find('div', class_='inline-block ratings-imdb-rating')['data-value']
        certification = container.find('span', class_='certificate').text.strip() if container.find('span', class_='certificate') else ''

        second_csv_data.append({
            'Movie Name': title,
            'Stars': ', '.join(star_names),
            'Votes': votes,
            'Genre': ', '.join(genres),
            'Gross collection': gross,
            'Popularity': popularity,
            'Certification': certification
        })

    page += 50


In [3]:
# Create a DataFrame for the first CSV data
first_csv_df = pd.DataFrame(first_csv_data)

# Create a DataFrame for the second CSV data
second_csv_df = pd.DataFrame(second_csv_data)

In [4]:
first_csv_df.head(5)

,Sno,Movie Name,Director Name,Duration,Year,Ratings,Metascore
0,1,Spider-Man: Across the Spider-Verse,"Joaquim Dos Santos, Kemp Powers, Justin K. Tho...",140,2023,9.0,86
1,2,The Dark Knight,"Christopher Nolan, Christian Bale, Heath Ledge...",152,2008,9.0,84
2,3,The Lord of the Rings: The Return of the King,"Peter Jackson, Elijah Wood, Viggo Mortensen, I...",201,2003,9.0,94
3,4,Inception,"Christopher Nolan, Leonardo DiCaprio, Joseph G...",148,2010,8.8,74
4,5,The Lord of the Rings: The Fellowship of the Ring,"Peter Jackson, Elijah Wood, Ian McKellen, Orla...",178,2001,8.8,92


In [5]:
# Split the 'Stars' column into separate columns
split_directors = first_csv_df['Director Name'].str.split(', ', expand=True)

# Get the maximum number of stars in any row
max_directors = split_directors.apply(lambda row: len(row.dropna()), axis=1).max()

# Create new columns for stars
director_columns = [f'Director{i}' for i in range(1, max_directors + 1)]
first_csv_df[director_columns] = split_directors.iloc[:, :max_directors]

In [6]:
first_csv_df.head(5)

,Sno,Movie Name,Director Name,Duration,Year,Ratings,Metascore,Director1,Director2,Director3,Director4,Director5,Director6,Director7,Director8,Director9,Director10
0,1,Spider-Man: Across the Spider-Verse,"Joaquim Dos Santos, Kemp Powers, Justin K. Tho...",140,2023,9.0,86,Joaquim Dos Santos,Kemp Powers,Justin K. Thompson,Shameik Moore,Hailee Steinfeld,Brian Tyree Henry,Luna Lauren Velez,None,None,None
1,2,The Dark Knight,"Christopher Nolan, Christian Bale, Heath Ledge...",152,2008,9.0,84,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,None,None,None,None,None
2,3,The Lord of the Rings: The Return of the King,"Peter Jackson, Elijah Wood, Viggo Mortensen, I...",201,2003,9.0,94,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,None,None,None,None,None
3,4,Inception,"Christopher Nolan, Leonardo DiCaprio, Joseph G...",148,2010,8.8,74,Christopher Nolan,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe,None,None,None,None,None
4,5,The Lord of the Rings: The Fellowship of the Ring,"Peter Jackson, Elijah Wood, Ian McKellen, Orla...",178,2001,8.8,92,Peter Jackson,Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean,None,None,None,None,None


In [7]:
second_csv_df.head(5)

,Movie Name,Stars,Votes,Genre,Gross collection,Popularity,Certification
0,Spider-Man: Across the Spider-Verse,"Kemp Powers, Justin K. Thompson, Shameik Moore...",127907,"Animation, Action, Adventure",127907,9,U
1,The Dark Knight,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",2725921,"Action, Crime, Drama","534,858,444",9,UA
2,The Lord of the Rings: The Return of the King,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",1891275,"Action, Adventure, Drama","377,845,905",9,U
3,Inception,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",2419368,"Action, Adventure, Sci-Fi","292,576,195",8.8,UA
4,The Lord of the Rings: The Fellowship of the Ring,"Elijah Wood, Ian McKellen, Orlando Bloom, Sean...",1919868,"Action, Adventure, Drama","315,544,750",8.8,U


In [8]:
first_csv_df.drop("Director Name", axis=1, inplace=True)

In [9]:
first_csv_df.head(5)

,Sno,Movie Name,Duration,Year,Ratings,Metascore,Director1,Director2,Director3,Director4,Director5,Director6,Director7,Director8,Director9,Director10
0,1,Spider-Man: Across the Spider-Verse,140,2023,9.0,86,Joaquim Dos Santos,Kemp Powers,Justin K. Thompson,Shameik Moore,Hailee Steinfeld,Brian Tyree Henry,Luna Lauren Velez,None,None,None
1,2,The Dark Knight,152,2008,9.0,84,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,None,None,None,None,None
2,3,The Lord of the Rings: The Return of the King,201,2003,9.0,94,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,None,None,None,None,None
3,4,Inception,148,2010,8.8,74,Christopher Nolan,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe,None,None,None,None,None
4,5,The Lord of the Rings: The Fellowship of the Ring,178,2001,8.8,92,Peter Jackson,Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean,None,None,None,None,None


In [10]:
first_csv_df.to_csv("directors_info.csv")

In [11]:
# Split the 'Stars' column into separate columns
split_stars = second_csv_df['Stars'].str.split(', ', expand=True)

# Get the maximum number of stars in any row
max_stars = split_stars.apply(lambda row: len(row.dropna()), axis=1).max()

# Create new columns for stars
star_columns = [f'Star{i}' for i in range(1, max_stars + 1)]
second_csv_df[star_columns] = split_stars.iloc[:, :max_stars]

In [12]:
second_csv_df.head(5)

,Movie Name,Stars,Votes,Genre,Gross collection,Popularity,Certification,Star1,Star2,Star3,Star4,Star5,Star6,Star7,Star8,Star9
0,Spider-Man: Across the Spider-Verse,"Kemp Powers, Justin K. Thompson, Shameik Moore...",127907,"Animation, Action, Adventure",127907,9,U,Kemp Powers,Justin K. Thompson,Shameik Moore,Hailee Steinfeld,Brian Tyree Henry,Luna Lauren Velez,None,None,None
1,The Dark Knight,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",2725921,"Action, Crime, Drama","534,858,444",9,UA,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,None,None,None,None,None
2,The Lord of the Rings: The Return of the King,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",1891275,"Action, Adventure, Drama","377,845,905",9,U,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,None,None,None,None,None
3,Inception,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",2419368,"Action, Adventure, Sci-Fi","292,576,195",8.8,UA,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe,None,None,None,None,None
4,The Lord of the Rings: The Fellowship of the Ring,"Elijah Wood, Ian McKellen, Orlando Bloom, Sean...",1919868,"Action, Adventure, Drama","315,544,750",8.8,U,Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean,None,None,None,None,None


In [13]:
# Split the 'Genre' column into separate columns
split_genres = second_csv_df['Genre'].str.split(', ', expand=True)

# Get the maximum number of stars in any row
max_genres = split_genres.apply(lambda row: len(row.dropna()), axis=1).max()

# Create new columns for stars
genre_columns = [f'Genre{i}' for i in range(1, max_genres + 1)]
second_csv_df[genre_columns] = split_genres.iloc[:, :max_genres]

In [14]:
second_csv_df.head(5)

,Movie Name,Stars,Votes,Genre,Gross collection,Popularity,Certification,Star1,Star2,Star3,Star4,Star5,Star6,Star7,Star8,Star9,Genre1,Genre2,Genre3
0,Spider-Man: Across the Spider-Verse,"Kemp Powers, Justin K. Thompson, Shameik Moore...",127907,"Animation, Action, Adventure",127907,9,U,Kemp Powers,Justin K. Thompson,Shameik Moore,Hailee Steinfeld,Brian Tyree Henry,Luna Lauren Velez,None,None,None,Animation,Action,Adventure
1,The Dark Knight,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",2725921,"Action, Crime, Drama","534,858,444",9,UA,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,None,None,None,None,None,Action,Crime,Drama
2,The Lord of the Rings: The Return of the King,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",1891275,"Action, Adventure, Drama","377,845,905",9,U,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,None,None,None,None,None,Action,Adventure,Drama
3,Inception,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",2419368,"Action, Adventure, Sci-Fi","292,576,195",8.8,UA,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe,None,None,None,None,None,Action,Adventure,Sci-Fi
4,The Lord of the Rings: The Fellowship of the Ring,"Elijah Wood, Ian McKellen, Orlando Bloom, Sean...",1919868,"Action, Adventure, Drama","315,544,750",8.8,U,Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean,None,None,None,None,None,Action,Adventure,Drama


In [15]:
second_csv_df.drop(["Stars", "Genre"], axis=1, inplace=True)

In [16]:
second_csv_df.head(5)

,Movie Name,Votes,Gross collection,Popularity,Certification,Star1,Star2,Star3,Star4,Star5,Star6,Star7,Star8,Star9,Genre1,Genre2,Genre3
0,Spider-Man: Across the Spider-Verse,127907,127907,9,U,Kemp Powers,Justin K. Thompson,Shameik Moore,Hailee Steinfeld,Brian Tyree Henry,Luna Lauren Velez,None,None,None,Animation,Action,Adventure
1,The Dark Knight,2725921,"534,858,444",9,UA,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,None,None,None,None,None,Action,Crime,Drama
2,The Lord of the Rings: The Return of the King,1891275,"377,845,905",9,U,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,None,None,None,None,None,Action,Adventure,Drama
3,Inception,2419368,"292,576,195",8.8,UA,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe,None,None,None,None,None,Action,Adventure,Sci-Fi
4,The Lord of the Rings: The Fellowship of the Ring,1919868,"315,544,750",8.8,U,Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean,None,None,None,None,None,Action,Adventure,Drama


In [17]:
second_csv_df.to_csv("stars_info.csv")

## Phase-2

### Make two tables and corresponding columns provided in the above CSV files in SQLite DBMS. Insert all data of each CSV file in each of the created tables. Now start querying the table(s) in the  SQL workbench / SQLite database :
#### Table 1: Sno, MovieName, Director Name, Duration, genre, ratings
1)  Display all the details of movies created by directors Christopher and Matt Reeves.
2) Display all the details of movies with a duration of 140 minutes to 190 minutes.
3) Display all details of movies with ratings above 7 in ascending order.
4) Display all movie names in descending order.
5) Display movie name starts with ‘P’ and their rating is greater than 7.


#### Table 2: Movie Name, stars, votes, Genre, Gross collection, popularity, Certification
1) Display all movie names with star Arnold Schwarzenegger in ascending order.
2    2) Display all details of the movie with the highest number of votes.
3) Display movie names with gross collections in descending order.
4) Display the gross collection of movies with the star Arnold.
5) Display all details of movies with comedy and action genres.

#### Make subquery :
1) Display all details from both tables where movie names are the same.
2)   Display all movie names, Director, ratings, and gross collection where the genre is action.
3)      Display all details from both tables with the highest gross collection.
4)      Display all details from both tables with the highest ratings
5)      Display all details from both tables with the lowest gross collection and lowest ratings

#### Now once completed with queries in the SQLite database, then make the exact query solutions by using PANDAS SQL in the data frame.  Load the CSV data in a data frame and start making solutions for all the above 15 queries using PANDAS SQL. You may use concat or merge joins per the requirements basis to make 5 join queries.


In [108]:
# Import necessary Libraries
import sqlite3
from sqlite3 import Error

2.6.0


In [122]:
# Create Sqllite database
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

conn = create_connection(r"C:\Users\TCS\Desktop\Project\Capstone IMDB\imdb.db")
conn.close()

In [22]:
conn = create_connection(r"C:\Users\TCS\Desktop\Project\Capstone IMDB\imdb.db")
cur = conn.cursor()

In [135]:
# Create directors table

qry = """CREATE TABLE directors (
    Sno INTEGER,
    MovieName TEXT,
    Duration INTEGER,
    Year INTEGER,
    Ratings REAL,
    Metascore INTEGER,
    Director1 TEXT,
    Director2 TEXT,
    Director3 TEXT,
    Director4 TEXT,
    Director5 TEXT,
    Director6 TEXT,
    Director7 TEXT,
    Director8 TEXT,
    Director9 TEXT,
    Director10 TEXT
);
"""
cur.execute(qry)

In [133]:
directors_data = pd.read_csv("directors_info.csv", header=0)

In [134]:
directors_data.head(5)

,Sno,Movie Name,Duration,Year,Ratings,Metascore,Director1,Director2,Director3,Director4,Director5,Director6,Director7,Director8,Director9,Director10
0,1,Spider-Man: Across the Spider-Verse,140,2023,9.0,86.0,Joaquim Dos Santos,Kemp Powers,Justin K. Thompson,Shameik Moore,Hailee Steinfeld,Brian Tyree Henry,Luna Lauren Velez,NaN,NaN,NaN
1,2,The Dark Knight,152,2008,9.0,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,NaN,NaN,NaN,NaN,NaN
2,3,The Lord of the Rings: The Return of the King,201,2003,9.0,94.0,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,NaN,NaN,NaN,NaN,NaN
3,4,Inception,148,2010,8.8,74.0,Christopher Nolan,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe,NaN,NaN,NaN,NaN,NaN
4,5,The Lord of the Rings: The Fellowship of the Ring,178,2001,8.8,92.0,Peter Jackson,Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean,NaN,NaN,NaN,NaN,NaN


In [137]:
# Insert directors data into table
directors_data.to_sql('directors', conn, if_exists='replace', index=False)

C:\Users\TCS\anaconda3 amita\lib\site-packages\pandas\core\generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [139]:
# Create stars table

qry = """CREATE TABLE stars (
    MovieName TEXT,
    Votes INTEGER,
    GrossCollection REAL,
    Popularity REAL,
    Certification TEXT,
    Star1 TEXT,
    Star2 TEXT,
    Star3 TEXT,
    Star4 TEXT,
    Star5 TEXT,
    Star6 TEXT,
    Star7 TEXT,
    Star8 TEXT,
    Star9 TEXT,
    Genre1 TEXT,
    Genre2 TEXT,
    Genre3 TEXT
);
"""
cur.execute(qry)

In [138]:
stars_data = pd.read_csv("stars_info.csv", header=0)

In [140]:
# Insert stars data into table
stars_data.to_sql('stars', conn, if_exists='replace', index=False)

In [151]:
#1)  Display all the details of movies created by directors Christopher and Matt Reeves
query = "SELECT * FROM directors WHERE Director1 IN ('Christopher Nolan', 'Matt Reeves') \
         OR Director2 IN ('Christopher Nolan', 'Matt Reeves') \
         OR Director3 IN ('Christopher Nolan', 'Matt Reeves') \
         OR Director4 IN ('Christopher Nolan', 'Matt Reeves') \
         OR Director5 IN ('Christopher Nolan', 'Matt Reeves') \
         OR Director6 IN ('Christopher Nolan', 'Matt Reeves') \
         OR Director7 IN ('Christopher Nolan', 'Matt Reeves') \
         OR Director8 IN ('Christopher Nolan', 'Matt Reeves') \
         OR Director9 IN ('Christopher Nolan', 'Matt Reeves');"
df = pd.read_sql_query(query, conn)
df

,Sno,Movie Name,Duration,Year,Ratings,Metascore,Director1,Director2,Director3,Director4,Director5,Director6,Director7,Director8,Director9,Director10
0,2,The Dark Knight,152,2008,9.0,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,None,None,None,None,None
1,4,Inception,148,2010,8.8,74.0,Christopher Nolan,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe,None,None,None,None,None
2,22,The Dark Knight Rises,164,2012,8.4,78.0,Christopher Nolan,Christian Bale,Tom Hardy,Anne Hathaway,Gary Oldman,None,None,None,None,None
3,42,Batman Begins,140,2005,8.2,70.0,Christopher Nolan,Christian Bale,Michael Caine,Ken Watanabe,Liam Neeson,None,None,None,None,None
4,131,The Batman,176,2022,7.8,72.0,Matt Reeves,Robert Pattinson,Zoë Kravitz,Jeffrey Wright,Colin Farrell,None,None,None,None,None
5,137,Dunkirk,106,2017,7.8,94.0,Christopher Nolan,Fionn Whitehead,Barry Keoghan,Mark Rylance,Tom Hardy,None,None,None,None,None
6,225,Dawn of the Planet of the Apes,130,2014,7.6,79.0,Matt Reeves,Gary Oldman,Keri Russell,Andy Serkis,Kodi Smit-McPhee,None,None,None,None,None
7,305,War for the Planet of the Apes,140,2017,7.4,82.0,Matt Reeves,Andy Serkis,Woody Harrelson,Steve Zahn,Karin Konoval,None,None,None,None,None
8,322,Tenet,150,2020,7.3,69.0,Christopher Nolan,John David Washington,Robert Pattinson,Elizabeth Debicki,Juhan Ulfsak,None,None,None,None,None
9,502,Cloverfield,85,2008,7.0,64.0,Matt Reeves,Mike Vogel,Jessica Lucas,Lizzy Caplan,T.J. Miller,None,None,None,None,None


In [152]:
#2) Display all the details of movies with a duration of 140 minutes to 190 minutes.
query = "SELECT * FROM directors WHERE Duration >= 140 and Duration <= 190;"
df = pd.read_sql_query(query, conn)
df

,Sno,Movie Name,Duration,Year,Ratings,Metascore,Director1,Director2,Director3,Director4,Director5,Director6,Director7,Director8,Director9,Director10
0,1,Spider-Man: Across the Spider-Verse,140,2023,9.0,86.0,Joaquim Dos Santos,Kemp Powers,Justin K. Thompson,Shameik Moore,Hailee Steinfeld,Brian Tyree Henry,Luna Lauren Velez,None,None,None
1,2,The Dark Knight,152,2008,9.0,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,None,None,None,None,None
2,4,Inception,148,2010,8.8,74.0,Christopher Nolan,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe,None,None,None,None,None
3,5,The Lord of the Rings: The Fellowship of the Ring,178,2001,8.8,92.0,Peter Jackson,Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean,None,None,None,None,None
4,6,The Lord of the Rings: The Two Towers,179,2002,8.8,87.0,Peter Jackson,Elijah Wood,Ian McKellen,Viggo Mortensen,Orlando Bloom,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,1740,Gunday,152,2014,2.7,NaN,Ali Abbas Zafar,Ranveer Singh,Arjun Kapoor,Priyanka Chopra Jonas,Irrfan Khan,None,None,None,None,None
200,1741,Liger,140,2022,2.7,NaN,Puri Jagannadh,Mike Tyson,Vijay Deverakonda,Ananya Panday,Ramya Krishnan,None,None,None,None,None
201,1744,Heropanti 2,145,2022,2.5,NaN,Ahmed Khan,Tiger Shroff,Nawazuddin Siddiqui,Tara Sutaria,Amber Doig-Thorne,None,None,None,None,None
202,1745,Laxmii,141,2020,2.5,NaN,Raghava Lawrence,Balakrishnan Thevar,Akshay Kumar,Kiara Advani,Sharad Kelkar,Ashwini Kalsekar,None,None,None,None


In [156]:
#3) Display all the details of movies with ratings above 7 in ascending order.
query = "SELECT * FROM directors WHERE Ratings > 7 ORDER BY Ratings ASC;"
df = pd.read_sql_query(query, conn)
df

,Sno,Movie Name,Duration,Year,Ratings,Metascore,Director1,Director2,Director3,Director4,Director5,Director6,Director7,Director8,Director9,Director10
0,430,Fast & Furious 7,137,2015,7.1,67.0,James Wan,Vin Diesel,Paul Walker,Dwayne Johnson,Jason Statham,None,None,None,None,None
1,431,The Mummy,124,1999,7.1,48.0,Stephen Sommers,Brendan Fraser,Rachel Weisz,John Hannah,Arnold Vosloo,None,None,None,None,None
2,432,Wrath of Man,119,2021,7.1,57.0,Guy Ritchie,Jason Statham,Holt McCallany,Josh Hartnett,Rocci Boy Williams,None,None,None,None,None
3,433,Mission: Impossible,110,1996,7.1,59.0,Brian De Palma,Tom Cruise,Jon Voight,Emmanuelle Béart,Henry Czerny,None,None,None,None,None
4,434,Free Guy,115,2021,7.1,62.0,Shawn Levy,Ryan Reynolds,Jodie Comer,Taika Waititi,Lil Rel Howery,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,5,The Lord of the Rings: The Fellowship of the Ring,178,2001,8.8,92.0,Peter Jackson,Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean,None,None,None,None,None
484,6,The Lord of the Rings: The Two Towers,179,2002,8.8,87.0,Peter Jackson,Elijah Wood,Ian McKellen,Viggo Mortensen,Orlando Bloom,None,None,None,None,None
485,1,Spider-Man: Across the Spider-Verse,140,2023,9.0,86.0,Joaquim Dos Santos,Kemp Powers,Justin K. Thompson,Shameik Moore,Hailee Steinfeld,Brian Tyree Henry,Luna Lauren Velez,None,None,None
486,2,The Dark Knight,152,2008,9.0,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,None,None,None,None,None


In [162]:
#4) Display all movie names in descending order.
query = "SELECT * FROM directors ORDER BY 'Movie Name' DESC;"
df = pd.read_sql_query(query, conn)
df

,Sno,Movie Name,Duration,Year,Ratings,Metascore,Director1,Director2,Director3,Director4,Director5,Director6,Director7,Director8,Director9,Director10
0,1,Spider-Man: Across the Spider-Verse,140,2023,9.0,86.0,Joaquim Dos Santos,Kemp Powers,Justin K. Thompson,Shameik Moore,Hailee Steinfeld,Brian Tyree Henry,Luna Lauren Velez,None,None,None
1,2,The Dark Knight,152,2008,9.0,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,None,None,None,None,None
2,3,The Lord of the Rings: The Return of the King,201,2003,9.0,94.0,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,None,None,None,None,None
3,4,Inception,148,2010,8.8,74.0,Christopher Nolan,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe,None,None,None,None,None
4,5,The Lord of the Rings: The Fellowship of the Ring,178,2001,8.8,92.0,Peter Jackson,Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1748,1749,Radhe,109,2021,1.9,NaN,Prabhu Deva,Salman Khan,Disha Patani,Randeep Hooda,Jackie Shroff,None,None,None,None,None
1749,1750,Race 3,160,2018,1.9,NaN,Remo D'Souza,Anil Kapoor,Salman Khan,Bobby Deol,Jacqueline Fernandez,None,None,None,None,None
1750,1751,Angels Apocalypse,85,2015,1.6,NaN,Sean Cain,Enzo Zelocchi,Enzo Zelocchi,Jana Rochelle,Ryan C.F. Buckley,William Kirkham,None,None,None,None
1751,1752,Elk*rtuk,125,2021,1.5,NaN,Keith English,Vivianne Bánovits,András Mózes,Barna Bokor,Gabriella Gubás,None,None,None,None,None


In [166]:
#5) Display movie name starts with ‘P’ and their rating is greater than 7.
query = "SELECT * FROM directors WHERE `Movie Name` LIKE 'P%' AND Ratings > 7;"
df = pd.read_sql_query(query, conn)
df


,Sno,Movie Name,Duration,Year,Ratings,Metascore,Director1,Director2,Director3,Director4,Director5,Director6,Director7,Director8,Director9,Director10
0,57,Paan Singh Tomar,135,2012,8.2,NaN,Tigmanshu Dhulia,Irrfan Khan,Mahie Gill,Rajesh Abhay,Hemendra Dandotiya,None,None,None,None,None
1,59,Pirates of the Caribbean: The Curse of the Bla...,143,2003,8.1,63.0,Gore Verbinski,Johnny Depp,Geoffrey Rush,Orlando Bloom,Keira Knightley,None,None,None,None,None
2,119,Per un pugno di dollari,99,1964,7.9,65.0,Sergio Leone,Clint Eastwood,Gian Maria Volontè,Marianne Koch,Wolfgang Lukschy,None,None,None,None,None
3,142,Predator,107,1987,7.8,47.0,John McTiernan,Arnold Schwarzenegger,Carl Weathers,Kevin Peter Hall,Elpidia Carrillo,None,None,None,None,None
4,192,Ponniyin Selvan: I,167,2022,7.7,NaN,Mani Ratnam,Vikram,Aishwarya Rai Bachchan,Jayam Ravi,Karthi,None,None,None,None,None
5,238,Pushpa: The Rise - Part 1,179,2021,7.6,NaN,Sukumar,Allu Arjun,Fahadh Faasil,Rashmika Mandanna,Jagadeesh Prathap Bandari,None,None,None,None,None
6,293,Predestination,97,I) (2014,7.4,69.0,Michael Spierig,Peter Spierig,Ethan Hawke,Sarah Snook,Noah Taylor,Madeleine West,None,None,None,None
7,339,Pirates of the Caribbean: Dead Man's Chest,151,2006,7.3,53.0,Gore Verbinski,Johnny Depp,Orlando Bloom,Keira Knightley,Jack Davenport,None,None,None,None,None
8,343,Patriots Day,133,2016,7.3,69.0,Peter Berg,Mark Wahlberg,Michelle Monaghan,J.K. Simmons,John Goodman,None,None,None,None,None
9,380,Point Break,122,1991,7.2,59.0,Kathryn Bigelow,Patrick Swayze,Keanu Reeves,Gary Busey,Lori Petty,None,None,None,None,None


In [171]:
#1)  Display all movie names with star Arnold Schwarzenegger in ascending order
query = "SELECT * FROM stars WHERE star1 IN ('Arnold Schwarzenegger') \
         OR star2 IN ('Arnold Schwarzenegger') \
         OR star3 IN ('Arnold Schwarzenegger') \
         OR star4 IN ('Arnold Schwarzenegger') \
         OR star5 IN ('Arnold Schwarzenegger') \
         OR star6 IN ('Arnold Schwarzenegger') \
         OR star7 IN ('Arnold Schwarzenegger') \
         OR star8 IN ('Arnold Schwarzenegger') \
         OR star9 IN ('Arnold Schwarzenegger') ORDER BY 'Ratings' ASC;"
df = pd.read_sql_query(query, conn)
df

,Movie Name,Votes,Gross collection,Popularity,Certification,Star1,Star2,Star3,Star4,Star5,Star6,Star7,Star8,Star9,Genre1,Genre2,Genre3
0,Terminator 2: Judgment Day,1124566,"204,843,350",8.6,A,Arnold Schwarzenegger,Linda Hamilton,Edward Furlong,Robert Patrick,None,None,None,None,None,Action,Sci-Fi,None
1,The Terminator,886572,"38,400,000",8.1,A,Arnold Schwarzenegger,Linda Hamilton,Michael Biehn,Paul Winfield,None,None,None,None,None,Action,Sci-Fi,None
2,Predator,435013,"59,735,548",7.8,A,Arnold Schwarzenegger,Carl Weathers,Kevin Peter Hall,Elpidia Carrillo,None,None,None,None,None,Action,Adventure,Horror
3,Total Recall,341210,"119,394,840",7.5,A,Arnold Schwarzenegger,Sharon Stone,Michael Ironside,Rachel Ticotin,None,None,None,None,None,Action,Adventure,Sci-Fi
4,True Lies,269561,"146,282,411",7.3,A,Arnold Schwarzenegger,Jamie Lee Curtis,Tom Arnold,Bill Paxton,None,None,None,None,None,Action,Comedy,Thriller
5,Conan the Barbarian,155773,"39,565,475",6.9,A,Arnold Schwarzenegger,James Earl Jones,Max von Sydow,Sandahl Bergman,None,None,None,None,None,Action,Adventure,Fantasy
6,Commando,167620,"35,100,000",6.7,A,Arnold Schwarzenegger,Rae Dawn Chong,Dan Hedaya,Vernon Wells,None,None,None,None,None,Action,Adventure,Thriller
7,Escape Plan,249927,"25,132,228",6.7,A,Sylvester Stallone,Arnold Schwarzenegger,50 Cent,Vincent D'Onofrio,None,None,None,None,None,Action,Thriller,None
8,The Running Man,159185,"38,122,105",6.6,A,Arnold Schwarzenegger,Maria Conchita Alonso,Yaphet Kotto,Jim Brown,None,None,None,None,None,Action,Sci-Fi,Thriller
9,Last Action Hero,157517,"50,016,394",6.4,U,Arnold Schwarzenegger,F. Murray Abraham,Art Carney,Charles Dance,None,None,None,None,None,Action,Adventure,Comedy


In [172]:
#2) Display all details of the movie with the highest number of votes.
query = "SELECT * FROM stars ORDER BY Votes DESC LIMIT 1;"
df = pd.read_sql_query(query, conn)
df

,Movie Name,Votes,Gross collection,Popularity,Certification,Star1,Star2,Star3,Star4,Star5,Star6,Star7,Star8,Star9,Genre1,Genre2,Genre3
0,The Dark Knight,2722695,"534,858,444",9.0,UA,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,None,None,None,None,None,Action,Crime,Drama


In [173]:
#3) Display movie names with gross collections in descending order
query = "SELECT * FROM stars ORDER BY `Gross collection` DESC;"
df = pd.read_sql_query(query, conn)
df

,Movie Name,Votes,Gross collection,Popularity,Certification,Star1,Star2,Star3,Star4,Star5,Star6,Star7,Star8,Star9,Genre1,Genre2,Genre3
0,Hitman's Wife's Bodyguard,99869,99869,6.1,A,Ryan Reynolds,Samuel L. Jackson,Salma Hayek,Antonio Banderas,None,None,None,None,None,Action,Comedy,Crime
1,Gangs of Wasseypur,99452,99452,8.2,A,Manoj Bajpayee,Nawazuddin Siddiqui,Tigmanshu Dhulia,Richa Chadha,None,None,None,None,None,Action,Comedy,Crime
2,Olympus Has Fallen,283770,"98,925,640",6.5,A,Gerard Butler,Aaron Eckhart,Morgan Freeman,Angela Bassett,None,None,None,None,None,Action,Thriller,None
3,The Green Hornet,162608,"98,780,042",5.8,18,Seth Rogen,Jay Chou,Christoph Waltz,Cameron Diaz,None,None,None,None,None,Action,Comedy,Crime
4,Sherlock Jr.,52562,"977,375",8.2,None,Buster Keaton,Kathryn McGuire,Joe Keaton,Erwin Connelly,None,None,None,None,None,Action,Comedy,Romance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1748,Love and Monsters,134625,"1,070,714",6.9,13,Dylan O'Brien,Jessica Henwick,Michael Rooker,Dan Ewing,None,None,None,None,None,Action,Adventure,Comedy
1749,The General,93990,"1,033,895",8.1,None,Buster Keaton,Buster Keaton,Marion Mack,Glen Cavender,Jim Farley,None,None,None,None,Action,Adventure,Comedy
1750,Attack the Block,109055,"1,024,175",6.7,R,John Boyega,Jodie Whittaker,Alex Esmail,Leeon Jones,None,None,None,None,None,Action,Adventure,Comedy
1751,Cowboy Bebop: Tengoku no tobira,50720,"1,000,045",7.8,U,Tensai Okamura,Hiroyuki Okiura,Yoshiyuki Takei,Beau Billingslea,Melissa Fahn,Nicholas Guest,Kôichi Yamadera,None,None,Animation,Action,Crime


In [189]:
#4) Display the gross collection of movies with the star Arnold.
 
query = "SELECT `Gross collection` FROM stars WHERE star1 LIKE '%Arnold%' OR star2 LIKE '%Arnold%' OR star3 LIKE '%Arnold%'"
df = pd.read_sql_query(query, conn)
df

,Gross collection
0,"204,843,350"
1,"38,400,000"
2,"59,735,548"
3,"119,394,840"
4,"146,282,411"
5,"39,565,475"
6,"35,100,000"
7,"25,132,228"
8,"38,122,105"
9,"50,016,394"


In [193]:
#5) Display all details of movies with comedy and action genres.

query = "SELECT * FROM stars WHERE Genre1 = 'Comedy' OR Genre2 = 'Comedy' OR Genre3 = 'Comedy' AND (Genre1 = 'Action' OR Genre2 = 'Action' OR Genre3 = 'Action')"
df = pd.read_sql_query(query, conn)
df

,Movie Name,Votes,Gross collection,Popularity,Certification,Star1,Star2,Star3,Star4,Star5,Star6,Star7,Star8,Star9,Genre1,Genre2,Genre3
0,Guardians of the Galaxy Vol. 3,164600,164600,8.2,UA,Chris Pratt,Chukwudi Iwuji,Bradley Cooper,Pom Klementieff,None,None,None,None,None,Action,Adventure,Comedy
1,Gangs of Wasseypur,99452,99452,8.2,A,Manoj Bajpayee,Nawazuddin Siddiqui,Tigmanshu Dhulia,Richa Chadha,None,None,None,None,None,Action,Comedy,Crime
2,Sherlock Jr.,52562,"977,375",8.2,None,Buster Keaton,Kathryn McGuire,Joe Keaton,Erwin Connelly,None,None,None,None,None,Action,Comedy,Romance
3,"Lock, Stock and Two Smoking Barrels",595814,"3,897,569",8.1,A,Jason Flemyng,Dexter Fletcher,Nick Moran,Jason Statham,None,None,None,None,None,Action,Comedy,Crime
4,The General,93990,"1,033,895",8.1,None,Buster Keaton,Buster Keaton,Marion Mack,Glen Cavender,Jim Farley,None,None,None,None,Action,Adventure,Comedy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,The Adventures of Pluto Nash,26472,"4,420,080",3.8,PG-13,Eddie Murphy,Jay Mohr,Randy Quaid,Rosario Dawson,None,None,None,None,None,Action,Comedy,Sci-Fi
367,The Adventures of Sharkboy and Lavagirl 3-D,40053,"39,177,684",3.7,U,Cayden Boyd,George Lopez,Kristin Davis,David Arquette,None,None,None,None,None,Action,Adventure,Comedy
368,Spy Kids 4: All the Time in the World,27006,"38,538,188",3.5,PG,Jessica Alba,Jeremy Piven,Joel McHale,Rowan Blanchard,None,None,None,None,None,Action,Adventure,Comedy
369,Heropanti 2,26997,26997,2.5,UA,Tiger Shroff,Nawazuddin Siddiqui,Tara Sutaria,Amber Doig-Thorne,None,None,None,None,None,Action,Comedy,Crime


### Make subquery :

In [201]:
#1) Display all details from both tables where movie names are the same.

query = """
SELECT *
FROM directors
INNER JOIN stars ON directors.`Movie Name` = stars.`Movie Name`
"""
df = pd.read_sql_query(query, conn)
df

,Sno,Movie Name,Duration,Year,Ratings,Metascore,Director1,Director2,Director3,Director4,...,Star3,Star4,Star5,Star6,Star7,Star8,Star9,Genre1,Genre2,Genre3
0,1,Spider-Man: Across the Spider-Verse,140,2023,9.0,86.0,Joaquim Dos Santos,Kemp Powers,Justin K. Thompson,Shameik Moore,...,Shameik Moore,Hailee Steinfeld,Brian Tyree Henry,Luna Lauren Velez,None,None,None,Animation,Action,Adventure
1,2,The Dark Knight,152,2008,9.0,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,...,Aaron Eckhart,Michael Caine,None,None,None,None,None,Action,Crime,Drama
2,3,The Lord of the Rings: The Return of the King,201,2003,9.0,94.0,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,...,Ian McKellen,Orlando Bloom,None,None,None,None,None,Action,Adventure,Drama
3,4,Inception,148,2010,8.8,74.0,Christopher Nolan,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,...,Elliot Page,Ken Watanabe,None,None,None,None,None,Action,Adventure,Sci-Fi
4,5,The Lord of the Rings: The Fellowship of the Ring,178,2001,8.8,92.0,Peter Jackson,Elijah Wood,Ian McKellen,Orlando Bloom,...,Orlando Bloom,Sean Bean,None,None,None,None,None,Action,Adventure,Drama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1810,1749,Radhe,109,2021,1.9,NaN,Prabhu Deva,Salman Khan,Disha Patani,Randeep Hooda,...,Randeep Hooda,Jackie Shroff,None,None,None,None,None,Action,Crime,Thriller
1811,1750,Race 3,160,2018,1.9,NaN,Remo D'Souza,Anil Kapoor,Salman Khan,Bobby Deol,...,Bobby Deol,Jacqueline Fernandez,None,None,None,None,None,Action,Crime,Thriller
1812,1751,Angels Apocalypse,85,2015,1.6,NaN,Sean Cain,Enzo Zelocchi,Enzo Zelocchi,Jana Rochelle,...,Jana Rochelle,Ryan C.F. Buckley,William Kirkham,None,None,None,None,Action,Fantasy,Sci-Fi
1813,1752,Elk*rtuk,125,2021,1.5,NaN,Keith English,Vivianne Bánovits,András Mózes,Barna Bokor,...,Barna Bokor,Gabriella Gubás,None,None,None,None,None,Action,Crime,Drama


In [205]:
# 2)   Display all movie names, Director, ratings, and gross collection where the genre is action.
query = """
SELECT directors.`Movie Name`, directors.Director1, directors.Director2, directors.Director3,
       directors.Director4, directors.Director5, directors.Director6, directors.Director7,
       directors.Director8, directors.Director9, directors.Ratings, stars.`Gross collection`
FROM directors
INNER JOIN stars ON directors.`Movie Name` = stars.`Movie Name`
WHERE stars.Genre1 = 'Action' OR stars.Genre2 = 'Action' OR stars.Genre3 = 'Action'
"""
df = pd.read_sql_query(query, conn)
df

,Movie Name,Director1,Director2,Director3,Director4,Director5,Director6,Director7,Director8,Director9,Ratings,Gross collection
0,Spider-Man: Across the Spider-Verse,Joaquim Dos Santos,Kemp Powers,Justin K. Thompson,Shameik Moore,Hailee Steinfeld,Brian Tyree Henry,Luna Lauren Velez,None,None,9.0,95163
1,The Dark Knight,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,None,None,None,None,9.0,"534,858,444"
2,The Lord of the Rings: The Return of the King,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,None,None,None,None,9.0,"377,845,905"
3,Inception,Christopher Nolan,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe,None,None,None,None,8.8,"292,576,195"
4,The Lord of the Rings: The Fellowship of the Ring,Peter Jackson,Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean,None,None,None,None,8.8,"315,544,750"
...,...,...,...,...,...,...,...,...,...,...,...,...
1810,Radhe,Prabhu Deva,Salman Khan,Disha Patani,Randeep Hooda,Jackie Shroff,None,None,None,None,1.9,178829
1811,Race 3,Remo D'Souza,Anil Kapoor,Salman Khan,Bobby Deol,Jacqueline Fernandez,None,None,None,None,1.9,"1,690,861"
1812,Angels Apocalypse,Sean Cain,Enzo Zelocchi,Enzo Zelocchi,Jana Rochelle,Ryan C.F. Buckley,William Kirkham,None,None,None,1.6,42914
1813,Elk*rtuk,Keith English,Vivianne Bánovits,András Mózes,Barna Bokor,Gabriella Gubás,None,None,None,None,1.5,39505


In [206]:
#3)      Display all details from both tables with the highest gross collection.
query = """
SELECT *
FROM Directors
INNER JOIN Stars ON Directors.`Movie Name` = Stars.`Movie Name`
WHERE Stars.`Gross collection` = (
    SELECT MAX(`Gross collection`)
    FROM Stars
)
"""
df = pd.read_sql_query(query, conn)
df

,Sno,Movie Name,Duration,Year,Ratings,Metascore,Director1,Director2,Director3,Director4,...,Star3,Star4,Star5,Star6,Star7,Star8,Star9,Genre1,Genre2,Genre3
0,1182,Hitman's Wife's Bodyguard,100,2021,6.1,32.0,Patrick Hughes,Ryan Reynolds,Samuel L. Jackson,Salma Hayek,...,Salma Hayek,Antonio Banderas,None,None,None,None,None,Action,Comedy,Crime


In [209]:
#4)      Display all details from both tables with the highest ratings
query = """
SELECT *
FROM Directors
INNER JOIN Stars ON Directors.`Movie Name` = Stars.`Movie Name`
WHERE Directors.Ratings = (
    SELECT MAX(Ratings)
    FROM Directors
)
"""

df = pd.read_sql_query(query, conn)
df

,Sno,Movie Name,Duration,Year,Ratings,Metascore,Director1,Director2,Director3,Director4,...,Star3,Star4,Star5,Star6,Star7,Star8,Star9,Genre1,Genre2,Genre3
0,1,Spider-Man: Across the Spider-Verse,140,2023,9.0,86.0,Joaquim Dos Santos,Kemp Powers,Justin K. Thompson,Shameik Moore,...,Shameik Moore,Hailee Steinfeld,Brian Tyree Henry,Luna Lauren Velez,None,None,None,Animation,Action,Adventure
1,2,The Dark Knight,152,2008,9.0,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,...,Aaron Eckhart,Michael Caine,None,None,None,None,None,Action,Crime,Drama
2,3,The Lord of the Rings: The Return of the King,201,2003,9.0,94.0,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,...,Ian McKellen,Orlando Bloom,None,None,None,None,None,Action,Adventure,Drama


In [213]:
#5)      Display all details from both tables with the lowest gross collection and lowest ratings
query = """
SELECT *
FROM Directors
INNER JOIN Stars ON Directors.`Movie Name` = Stars.`Movie Name`
WHERE Stars.`Gross collection` = (
    SELECT MIN(`Gross collection`)
    FROM Stars
)
OR Directors.Ratings = (
    SELECT MIN(Ratings)
    FROM Directors
)
"""

df = pd.read_sql_query(query, conn)
df

,Sno,Movie Name,Duration,Year,Ratings,Metascore,Director1,Director2,Director3,Director4,...,Star3,Star4,Star5,Star6,Star7,Star8,Star9,Genre1,Genre2,Genre3
0,1366,Army of the Dead,148,2021,5.8,57.0,Zack Snyder,Dave Bautista,Ella Purnell,Ana de la Reguera,...,Ana de la Reguera,Omari Hardwick,None,None,None,None,None,Action,Crime,Drama
1,1753,Sadak 2,133,2020,1.2,NaN,Mahesh Bhatt,Alia Bhatt,Priyanka Bose,Sanjay Dutt,...,Sanjay Dutt,Aditya Roy Kapoor,None,None,None,None,None,Action,Drama,None


In [214]:
query = """
SELECT *
FROM Directors
INNER JOIN Stars ON Directors.`Movie Name` = Stars.`Movie Name`
WHERE Stars.`Gross collection` = (
    SELECT MIN(`Gross collection`)
    FROM Stars
)
and Directors.Ratings = (
    SELECT MIN(Ratings)
    FROM Directors
)
"""

df = pd.read_sql_query(query, conn)
df

,Sno,Movie Name,Duration,Year,Ratings,Metascore,Director1,Director2,Director3,Director4,...,Star3,Star4,Star5,Star6,Star7,Star8,Star9,Genre1,Genre2,Genre3


### Query solutions using PANDAS SQL

In [215]:
import pandas as pd

In [227]:

# Read the directors.info CSV file into a DataFrame
directors_df = pd.read_csv(r"C:\Users\TCS\Desktop\Project\Capstone IMDB\directors_info.csv")



In [228]:
# Print the directors DataFrame
print(directors_df)

       Sno                                         Movie Name  Duration  Year  \
0        1                Spider-Man: Across the Spider-Verse       140  2023   
1        2                                    The Dark Knight       152  2008   
2        3      The Lord of the Rings: The Return of the King       201  2003   
3        4                                          Inception       148  2010   
4        5  The Lord of the Rings: The Fellowship of the Ring       178  2001   
...    ...                                                ...       ...   ...   
1748  1749                                              Radhe       109  2021   
1749  1750                                             Race 3       160  2018   
1750  1751                                  Angels Apocalypse        85  2015   
1751  1752                                           Elk*rtuk       125  2021   
1752  1753                                            Sadak 2       133  2020   

      Ratings  Metascore   

### Table 1: Sno, MovieName, Director Name, Duration, genre, ratings

In [235]:
#1)  Display all the details of movies created by directors Christopher and Matt Reeves.

# Filter the DataFrame based on the directors' names
filtered_directors_df = directors_df[
    (directors_df['Director1'] == 'Christopher Nolan') |
    (directors_df['Director2'] == 'Christopher Nolan') |
    (directors_df['Director3'] == 'Christopher Nolan') |
    (directors_df['Director4'] == 'Christopher Nolan') |
    (directors_df['Director5'] == 'Christopher Nolan') |
    (directors_df['Director6'] == 'Christopher Nolan') |
    (directors_df['Director7'] == 'Christopher Nolan') |
    (directors_df['Director8'] == 'Christopher Nolan') |
    (directors_df['Director9'] == 'Christopher Nolan') |
    (directors_df['Director1'] == 'Matt Reeves') |
    (directors_df['Director2'] == 'Matt Reeves') |
    (directors_df['Director3'] == 'Matt Reeves') |
    (directors_df['Director4'] == 'Matt Reeves') |
    (directors_df['Director5'] == 'Matt Reeves') |
    (directors_df['Director6'] == 'Matt Reeves') |
    (directors_df['Director7'] == 'Matt Reeves') |
    (directors_df['Director8'] == 'Matt Reeves') |
    (directors_df['Director9'] == 'Matt Reeves')
]

# Print the filtered DataFrame
print(filtered_directors_df)


     Sno                      Movie Name  Duration  Year  Ratings  Metascore  \
1      2                 The Dark Knight       152  2008      9.0       84.0   
3      4                       Inception       148  2010      8.8       74.0   
21    22           The Dark Knight Rises       164  2012      8.4       78.0   
41    42                   Batman Begins       140  2005      8.2       70.0   
130  131                      The Batman       176  2022      7.8       72.0   
136  137                         Dunkirk       106  2017      7.8       94.0   
224  225  Dawn of the Planet of the Apes       130  2014      7.6       79.0   
304  305  War for the Planet of the Apes       140  2017      7.4       82.0   
321  322                           Tenet       150  2020      7.3       69.0   
501  502                     Cloverfield        85  2008      7.0       64.0   

             Director1              Director2             Director3  \
1    Christopher Nolan         Christian Bale   

In [236]:
#2) Display all the details of movies with a duration of 140 minutes to 190 minutes.
# Filter the DataFrame based on the duration range
filtered_directors_df = directors_df[
    (directors_df['Duration'] >= 140) & (directors_df['Duration'] <= 190)
]

# Print the filtered DataFrame
print(filtered_directors_df)

       Sno                                         Movie Name  Duration  Year  \
0        1                Spider-Man: Across the Spider-Verse       140  2023   
1        2                                    The Dark Knight       152  2008   
3        4                                          Inception       148  2010   
4        5  The Lord of the Rings: The Fellowship of the Ring       178  2001   
5        6              The Lord of the Rings: The Two Towers       179  2002   
...    ...                                                ...       ...   ...   
1739  1740                                             Gunday       152  2014   
1740  1741                                              Liger       140  2022   
1743  1744                                        Heropanti 2       145  2022   
1744  1745                                             Laxmii       141  2020   
1749  1750                                             Race 3       160  2018   

      Ratings  Metascore   

In [237]:
# 3) Display all details of movies with ratings above 7 in ascending order.
# Filter the DataFrame based on ratings above 7
filtered_directors_df = directors_df[directors_df['Ratings'] > 7]

# Sort the DataFrame by ratings in ascending order
sorted_directors_df = filtered_directors_df.sort_values('Ratings', ascending=True)

# Print the sorted DataFrame
print(sorted_directors_df)

     Sno                                         Movie Name  Duration  Year  \
487  488                                     Gabbar Is Back       128  2015   
454  455                               Flags of Our Fathers       135  2006   
453  454                              Live Free or Die Hard       128  2007   
452  453                                  Deepwater Horizon       107  2016   
451  452                                          Desperado       104  1995   
..   ...                                                ...       ...   ...   
4      5  The Lord of the Rings: The Fellowship of the Ring       178  2001   
3      4                                          Inception       148  2010   
2      3      The Lord of the Rings: The Return of the King       201  2003   
1      2                                    The Dark Knight       152  2008   
0      1                Spider-Man: Across the Spider-Verse       140  2023   

     Ratings  Metascore                  Director1 

In [239]:
# 4) Display all movie names in descending order.
# Sort the DataFrame by movie names in descending order
sorted_directors_df = directors_df.sort_values('Movie Name', ascending=False)

# Reset the index of the DataFrame
sorted_directors_df = sorted_directors_df.reset_index(drop=True)

# Select only the Movie Name column
movie_names = sorted_directors_df['Movie Name']

# Print the movie names in descending order
print(movie_names)

0                     Ölümlü Dünya
1                         Æon Flux
2          xXx: State of the Union
3       xXx: Return of Xander Cage
4                              xXx
                   ...            
1748                     12 Rounds
1749                     10,000 BC
1750               1 - Nenokkadine
1751                           '71
1752                     #Saraitda
Name: Movie Name, Length: 1753, dtype: object


In [240]:
# 5) Display movie name starts with ‘P’ and their rating is greater than 7.
# Filter the DataFrame based on movie names starting with 'P' and rating greater than 7
filtered_directors_df = directors_df[
    (directors_df['Movie Name'].str.startswith('P')) & (directors_df['Ratings'] > 7)
]

# Select only the Movie Name column
movie_names = filtered_directors_df['Movie Name']

# Print the movie names
print(movie_names)


56                                      Paan Singh Tomar
58     Pirates of the Caribbean: The Curse of the Bla...
118                              Per un pugno di dollari
141                                             Predator
191                                   Ponniyin Selvan: I
237                            Pushpa: The Rise - Part 1
292                                       Predestination
338           Pirates of the Caribbean: Dead Man's Chest
342                                         Patriots Day
379                                          Point Break
437                                                 Prey
441             Pirates of the Caribbean: At World's End
458                                        Planet Terror
469                                              Payback
Name: Movie Name, dtype: object


### Table 2: Movie Name, stars, votes, Genre, Gross collection, popularity, Certification

In [241]:
# Read the stars.info CSV file into a DataFrame
stars_df = pd.read_csv(r"C:\Users\TCS\Desktop\Project\Capstone IMDB\stars_info.csv")


In [242]:
# Print the stars DataFrame
print(stars_df)

                                             Movie Name    Votes  \
0                   Spider-Man: Across the Spider-Verse    95163   
1                                       The Dark Knight  2722695   
2         The Lord of the Rings: The Return of the King  1889533   
3                                             Inception  2416660   
4     The Lord of the Rings: The Fellowship of the Ring  1918116   
...                                                 ...      ...   
1748                                              Radhe   178829   
1749                                             Race 3    47600   
1750                                  Angels Apocalypse    42914   
1751                                           Elk*rtuk    39505   
1752                                            Sadak 2    96269   

     Gross collection  Popularity Certification              Star1  \
0               95163         9.0             U        Kemp Powers   
1         534,858,444         9.0          

In [250]:
# 1) Display all movie names with star Arnold Schwarzenegger in ascending order.


# Create a list of the Star columns
star_columns = ['Star1', 'Star2', 'Star3', 'Star4', 'Star5', 'Star6', 'Star7', 'Star8', 'Star9']

# Reshape the DataFrame using pd.melt()
melted_df = pd.melt(stars_df, id_vars=['Movie Name'], value_vars=star_columns, var_name='Star Number', value_name='Star Name')

# Filter out missing values in the 'Star Name' column
filtered_df = melted_df.dropna(subset=['Star Name'])

# Filter the DataFrame based on movies with Arnold Schwarzenegger
filtered_df = filtered_df[filtered_df['Star Name'].str.contains('Arnold Schwarzenegger', case=False)]

# Sort the movie names in ascending order
sorted_movie_names = filtered_df['Movie Name'].sort_values(ascending=True)

# Print the movie names in ascending order
for movie_name in sorted_movie_names:
    print(movie_name)

Batman & Robin
Collateral Damage
Commando
Conan the Barbarian
Conan the Destroyer
End of Days
Eraser
Escape Plan
Kindergarten Cop
Last Action Hero
Predator
Raw Deal
Red Heat
Red Sonja
Sabotage
Terminator 2: Judgment Day
Terminator 3: Rise of the Machines
Terminator Genisys
Terminator: Dark Fate
The 6th Day
The Last Stand
The Running Man
The Terminator
Total Recall
True Lies


In [251]:
#2) Display all details of the movie with the highest number of votes
# Find the row with the highest number of votes
max_votes_row = stars_df.loc[stars_df['Votes'].idxmax()]

# Print all the details of the movie with the highest number of votes
print(max_votes_row)

Movie Name          The Dark Knight
Votes                       2722695
Gross collection        534,858,444
Popularity                      9.0
Certification                    UA
Star1                Christian Bale
Star2                  Heath Ledger
Star3                 Aaron Eckhart
Star4                 Michael Caine
Star5                           NaN
Star6                           NaN
Star7                           NaN
Star8                           NaN
Star9                           NaN
Genre1                       Action
Genre2                        Crime
Genre3                        Drama
Name: 1, dtype: object


In [252]:
#3) Display movie names with gross collections in descending order.
# Sort the DataFrame by 'Gross collection' column in descending order
sorted_df = stars_df.sort_values('Gross collection', ascending=False)

# Display the movie names with gross collections in descending order
movie_names = sorted_df['Movie Name']
print(movie_names)

1181          Hitman's Wife's Bodyguard
47                   Gangs of Wasseypur
863                  Olympus Has Fallen
1399                   The Green Hornet
54                         Sherlock Jr.
                     ...               
585                   Love and Monsters
69                          The General
710                    Attack the Block
159     Cowboy Bebop: Tengoku no tobira
1365                   Army of the Dead
Name: Movie Name, Length: 1753, dtype: object


In [253]:
# 4) Display the gross collection of movies with the star Arnold
# Filter the DataFrame to include only rows where any of the Star columns contain "Arnold"
filtered_df = stars_df.loc[
    (stars_df['Star1'].str.contains('Arnold', na=False)) |
    (stars_df['Star2'].str.contains('Arnold', na=False)) |
    (stars_df['Star3'].str.contains('Arnold', na=False)) |
    (stars_df['Star4'].str.contains('Arnold', na=False)) |
    (stars_df['Star5'].str.contains('Arnold', na=False)) |
    (stars_df['Star6'].str.contains('Arnold', na=False)) |
    (stars_df['Star7'].str.contains('Arnold', na=False)) |
    (stars_df['Star8'].str.contains('Arnold', na=False)) |
    (stars_df['Star9'].str.contains('Arnold', na=False))
]

# Display the gross collection of movies with Arnold
gross_collections = filtered_df['Gross collection']
print(gross_collections)

10      204,843,350
61       38,400,000
141      59,735,548
252     119,394,840
323     146,282,411
430     155,247,825
565      39,565,475
672      35,100,000
701      25,132,228
753      38,122,105
917      50,016,394
918     202,019,785
996      89,760,956
1005     12,050,299
1011    150,371,112
1071     91,457,688
1073     62,253,077
1107    101,295,562
1212     34,994,648
1313     31,042,035
1329     34,604,280
1375     66,889,043
1452     10,508,518
1496     16,209,459
1543     40,077,257
1637      6,948,633
1726    107,325,195
Name: Gross collection, dtype: object


In [259]:
# 5) Display all details of movies with comedy and action genres.
# Filter the DataFrame to include only rows with comedy and action genres
filtered_df = stars_df[
    (stars_df['Genre1'] == 'Comedy') | (stars_df['Genre2'] == 'Comedy') | (stars_df['Genre3'] == 'Comedy')
    & (stars_df['Genre1'] == 'Action') | (stars_df['Genre2'] == 'Action') | (stars_df['Genre3'] == 'Action')
]

# Display all the details of movies with comedy and action genres
print(filtered_df)


                                       Movie Name   Votes Gross collection  \
0             Spider-Man: Across the Spider-Verse   95163            95163   
16              Spider-Man: Into the Spider-Verse  580076      190,241,310   
29                                  Mononoke-hime  407422        2,375,308   
36                 Guardians of the Galaxy Vol. 3  164600           164600   
45             Kimetsu no Yaiba: Mugen Ressha-Hen   64706       47,700,000   
...                                           ...     ...              ...   
1725                 The Adventures of Pluto Nash   26472        4,420,080   
1727  The Adventures of Sharkboy and Lavagirl 3-D   40053       39,177,684   
1733        Spy Kids 4: All the Time in the World   27006       38,538,188   
1743                                  Heropanti 2   26997            26997   
1744                                       Laxmii   57994            57994   

      Popularity Certification                Star1            

#### Make subquery :

In [261]:
# 1) Display all details from both tables where movie names are the same.
# Perform an inner join on the "Movie Name" column
merged_df = pd.merge(directors_df, stars_df, on='Movie Name', how='inner')

# Display all the details where movie names are the same
print(merged_df)

       Sno                                         Movie Name  Duration  Year  \
0        1                Spider-Man: Across the Spider-Verse       140  2023   
1        2                                    The Dark Knight       152  2008   
2        3      The Lord of the Rings: The Return of the King       201  2003   
3        4                                          Inception       148  2010   
4        5  The Lord of the Rings: The Fellowship of the Ring       178  2001   
...    ...                                                ...       ...   ...   
1810  1749                                              Radhe       109  2021   
1811  1750                                             Race 3       160  2018   
1812  1751                                  Angels Apocalypse        85  2015   
1813  1752                                           Elk*rtuk       125  2021   
1814  1753                                            Sadak 2       133  2020   

      Ratings  Metascore   

In [271]:
# 2)   Display all movie names, Director, ratings, and gross collection where the genre is action.

# Filter the stars DataFrame to include only rows with "Action" genre
action_movies = stars_df[
    (stars_df['Genre1'].str.lower() == 'action') |
    (stars_df['Genre2'].str.lower() == 'action') |
    (stars_df['Genre3'].str.lower() == 'action')
]

# Merge the directors and stars DataFrames on the 'Movie Name' column
merged_df = pd.merge(directors_df, action_movies, on='Movie Name', how='inner')

# Select the desired columns
desired_columns = ['Movie Name', 'Director1', 'Director2', 'Director3', 'Director4', 'Director5',
                   'Director6', 'Director7', 'Director8', 'Director9', 'Director10', 'Ratings', 'Gross collection']
result_df = merged_df[desired_columns]

# Display the result
print(result_df)


                                             Movie Name           Director1  \
0                   Spider-Man: Across the Spider-Verse  Joaquim Dos Santos   
1                                       The Dark Knight   Christopher Nolan   
2         The Lord of the Rings: The Return of the King       Peter Jackson   
3                                             Inception   Christopher Nolan   
4     The Lord of the Rings: The Fellowship of the Ring       Peter Jackson   
...                                                 ...                 ...   
1810                                              Radhe         Prabhu Deva   
1811                                             Race 3        Remo D'Souza   
1812                                  Angels Apocalypse           Sean Cain   
1813                                           Elk*rtuk       Keith English   
1814                                            Sadak 2        Mahesh Bhatt   

              Director2             Director3      

In [272]:
# 3)      Display all details from both tables with the highest gross collection

# Merge the directors and stars DataFrames on the 'Movie Name' column
merged_df = pd.merge(directors_df, stars_df, on='Movie Name', how='inner')

# Get the movie(s) with the highest gross collection
max_gross = merged_df['Gross collection'].max()
highest_gross_movies = merged_df[merged_df['Gross collection'] == max_gross]

# Display all the details for the movie(s) with the highest gross collection
print(highest_gross_movies)


       Sno                 Movie Name  Duration  Year  Ratings  Metascore  \
1254  1182  Hitman's Wife's Bodyguard       100  2021      6.1       32.0   

           Director1      Director2          Director3    Director4  ...  \
1254  Patrick Hughes  Ryan Reynolds  Samuel L. Jackson  Salma Hayek  ...   

            Star3             Star4 Star5 Star6 Star7 Star8  Star9  Genre1  \
1254  Salma Hayek  Antonio Banderas   NaN   NaN   NaN   NaN    NaN  Action   

      Genre2 Genre3  
1254  Comedy  Crime  

[1 rows x 32 columns]


In [273]:
# 4)      Display all details from both tables with the highest ratings
# Merge the directors and stars DataFrames on the 'Movie Name' column
merged_df = pd.merge(directors_df, stars_df, on='Movie Name', how='inner')

# Get the movie(s) with the highest ratings
max_ratings = merged_df['Ratings'].max()
highest_rated_movies = merged_df[merged_df['Ratings'] == max_ratings]

# Display all the details for the movie(s) with the highest ratings
print(highest_rated_movies)


   Sno                                     Movie Name  Duration  Year  \
0    1            Spider-Man: Across the Spider-Verse       140  2023   
1    2                                The Dark Knight       152  2008   
2    3  The Lord of the Rings: The Return of the King       201  2003   

   Ratings  Metascore           Director1       Director2           Director3  \
0      9.0       86.0  Joaquim Dos Santos     Kemp Powers  Justin K. Thompson   
1      9.0       84.0   Christopher Nolan  Christian Bale        Heath Ledger   
2      9.0       94.0       Peter Jackson     Elijah Wood     Viggo Mortensen   

       Director4  ...          Star3             Star4              Star5  \
0  Shameik Moore  ...  Shameik Moore  Hailee Steinfeld  Brian Tyree Henry   
1  Aaron Eckhart  ...  Aaron Eckhart     Michael Caine                NaN   
2   Ian McKellen  ...   Ian McKellen     Orlando Bloom                NaN   

               Star6 Star7 Star8  Star9     Genre1     Genre2     Genre3 

In [275]:
#5)      Display all details from both tables with the lowest gross collection and lowest ratings


# Merge the directors and stars DataFrames on the 'Movie Name' column
merged_df = pd.merge(directors_df, stars_df, on='Movie Name', how='inner')

# Get the movie(s) with the lowest gross collection and lowest ratings
min_gross = merged_df['Gross collection'].min()
min_ratings = merged_df['Ratings'].min()

lowest_gross_movies = merged_df[merged_df['Gross collection'] == min_gross]
lowest_ratings_movies = merged_df[merged_df['Ratings'] == min_ratings]

# Display all the details for the movie(s) with the lowest gross collection
print("Movies with the lowest gross collection:")
print(lowest_gross_movies)

# Display all the details for the movie(s) with the lowest ratings
print("Movies with the lowest ratings:")
print(lowest_ratings_movies)


Movies with the lowest gross collection:
       Sno        Movie Name  Duration  Year  Ratings  Metascore    Director1  \
1435  1366  Army of the Dead       148  2021      5.8       57.0  Zack Snyder   

          Director2     Director3          Director4  ...              Star3  \
1435  Dave Bautista  Ella Purnell  Ana de la Reguera  ...  Ana de la Reguera   

               Star4 Star5 Star6 Star7 Star8  Star9  Genre1  Genre2 Genre3  
1435  Omari Hardwick   NaN   NaN   NaN   NaN    NaN  Action   Crime  Drama  

[1 rows x 32 columns]
Movies with the lowest ratings:
       Sno Movie Name  Duration  Year  Ratings  Metascore     Director1  \
1814  1753    Sadak 2       133  2020      1.2        NaN  Mahesh Bhatt   

       Director2      Director3    Director4  ...        Star3  \
1814  Alia Bhatt  Priyanka Bose  Sanjay Dutt  ...  Sanjay Dutt   

                  Star4 Star5 Star6 Star7 Star8  Star9  Genre1  Genre2 Genre3  
1814  Aditya Roy Kapoor   NaN   NaN   NaN   NaN    NaN  Action